In [ ]:
import cv2
import numpy as np

# 1. Load and crop (ROI)
img = cv2.imread('digit.jpg')

# Adjust ROI values as needed (x, y, w, h)
x, y, w, h = 757, 378, 780, 700
roi = img[y:y+h, x:x+w]

# 2. Grayscale
gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

# 3. Noise Reduction
blurred = cv2.GaussianBlur(gray, (5,5), 0)

# 4. Thresholding (Otsu)
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 5. Edge Detection
edges = cv2.Canny(thresh, 50, 150)

# 6. Contour Detection
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy for drawing
contour_img = roi.copy()

# Filter and draw contours
for c in contours:
    area = cv2.contourArea(c)
    if area > 50:  # filter small noise, adjust threshold as needed
        x_c, y_c, w_c, h_c = cv2.boundingRect(c)
        # Draw bounding boxes
        cv2.rectangle(contour_img, (x_c, y_c), (x_c + w_c, y_c + h_c), (0, 0, 255), 2)
        # Optional: mark center
        cx = x_c + w_c//2
        cy = y_c + h_c//2
        cv2.circle(contour_img, (cx, cy), 3, (0, 255, 0), -1)

# --- Display results ---
cv2.imshow('ROI', roi)
cv2.imshow('Threshold', thresh)
cv2.imshow('Contours', contour_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
pip install tensorflow


  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached grpcio-1.73.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using c

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32')/255.0
x_test = x_test.reshape(-1,28,28,1).astype('float32')/255.0

# Build a simple CNN
model = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(x_train,y_train,epochs=3,validation_split=0.1)
model.evaluate(x_test,y_test)
model.save('digit_cnn.h5')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 35s 18ms/step - accuracy: 0.8977 - loss: 0.3325 - val_accuracy: 0.9763 - val_loss: 0.0815
Epoch 2/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 38s 23ms/step - accuracy: 0.9823 - loss: 0.0567 - val_accuracy: 0.9877 - val_loss: 0.0432
Epoch 3/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 29s 17ms/step - accuracy: 0.9889 - loss: 0.0348 - val_accuracy: 0.9882 - val_loss: 0.0439
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9819 - loss: 0.0473


In [4]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2

# Load your saved CNN model (trained on 28x28 digits)
model = load_model('digit_cnn.h5')

detected_digits = []

for d in digit_images:  # digit_images is the list you built earlier
    # --- Preprocessing ---
    # Resize to 28x28
    d_resized = cv2.resize(d, (28, 28), interpolation=cv2.INTER_AREA)

    # Invert colors if necessary (white digit on black background)
    # Check visually: if your digits are already white on black, skip this
    d_resized = 255 - d_resized

    # Normalize to 0-1
    d_norm = d_resized.astype('float32') / 255.0

    # Reshape to (1,28,28,1) for CNN
    d_input = d_norm.reshape(1, 28, 28, 1)

    # --- Prediction ---
    pred = model.predict(d_input, verbose=0)  # probabilities
    label = int(np.argmax(pred))              # highest-probability class

    detected_digits.append(label)

print("Predicted digits (unsorted):", detected_digits)


NameError: name 'digit_images' is not defined

In [5]:
import tensorflow as tf

img_height = 28
img_width = 28

train_ds = tf.keras.utils.image_dataset_from_directory(
    "DIGIT",
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=32,
    label_mode='int'  # gives you integer labels 0–9
)

# Optional: normalize pixels from [0,255] to [0,1]
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))


Found 10 files belonging to 10 classes.


In [6]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, epochs=10)

# Save the model in new Keras format
model.save('digit_cnn.keras')
print("✅ Model saved as digit_cnn.keras")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 2.3157
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.4000 - loss: 2.2521
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.6000 - loss: 2.1950
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.6000 - loss: 2.1359
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7000 - loss: 2.0723
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.8000 - loss: 2.0009
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.8000 - loss: 1.9187
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.8000 - loss: 1.8251
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9000 - loss: 1.7213
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.9000 - loss: 1.6085
✅ Model saved as digit_cnn.keras


In [9]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

model = load_model('digit_cnn.keras')

img = cv2.imread('test3.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))
img = 255 - img  # invert if needed
img = img.astype('float32')/255.0
img = img.reshape(1, 28, 28, 1)

pred = model.predict(img)
print("Predicted digit:", np.argmax(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
Predicted digit: 6


In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# --- Load trained model ---
model = load_model('digit_cnn.keras')

# --- Load image and preprocess (same as before) ---
img = cv2.imread('digit.jpg')
x, y, w, h = 757, 378, 780, 700  # your ROI
roi = img[y:y+h, x:x+w]

gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5,5), 0)
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# --- Find contours ---
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# --- Sort contours left to right ---
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

# --- Copy image for drawing ---
annotated = roi.copy()

# --- Loop through contours ---
for c in contours:
    area = cv2.contourArea(c)
    if area > 50:  # filter small noise
        x_c, y_c, w_c, h_c = cv2.boundingRect(c)

        # Crop digit patch
        digit_patch = thresh[y_c:y_c+h_c, x_c:x_c+w_c]

        # Preprocess for model
        d_resized = cv2.resize(digit_patch, (28,28))
        d_resized = 255 - d_resized  # invert if needed
        d_norm = d_resized.astype('float32')/255.0
        d_input = d_norm.reshape(1,28,28,1)

        # Predict
        pred = model.predict(d_input, verbose=0)
        label = int(np.argmax(pred))

        # Draw rectangle on annotated image
        cv2.rectangle(annotated, (x_c, y_c), (x_c+w_c, y_c+h_c), (0,255,0), 2)

        # Put label text
        cv2.putText(annotated, str(label), (x_c, y_c-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

# --- Show final annotated image ---
cv2.imshow('Detected Digits', annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# --- Load trained CNN model ---
model = load_model('digit_cnn.keras')

# --- Load and crop ROI ---
img = cv2.imread('test3.png')
#x, y, w, h = 757, 378, 780, 700  # adjust to your ROI
roi = img

# --- Preprocess ---
gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (9,9), 0)
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# ✅ Morphological closing to merge segments
kernel = np.ones((7,7), np.uint8)# tune size if needed
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# --- Find contours on closed image ---
contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours left-to-right
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

# Copy for drawing
annotated = roi.copy()

detected_digits = []

for c in contours:
    area = cv2.contourArea(c)
    if area > 100:  # filter tiny noise
        x_c, y_c, w_c, h_c = cv2.boundingRect(c)

        # Crop the digit region
        digit_patch = thresh[y_c:y_c+h_c, x_c:x_c+w_c]

        # Preprocess for model
        d_resized = cv2.resize(digit_patch, (28,28), interpolation=cv2.INTER_AREA)
        d_resized = 255 - d_resized  # invert if needed
        d_norm = d_resized.astype('float32') / 255.0
        d_input = d_norm.reshape(1, 28, 28, 1)

        # Predict with CNN
        pred = model.predict(d_input, verbose=0)
        label = int(np.argmax(pred))
        detected_digits.append(label)

        # Draw rectangle and label
        cv2.rectangle(annotated, (x_c, y_c), (x_c+w_c, y_c+h_c), (0,255,0), 2)
        cv2.putText(annotated, str(label), (x_c, y_c-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

# Show results
cv2.imshow('Closed', closed)         # see merged mask
cv2.imshow('Detected Digits', annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Detected digits:", detected_digits)


Detected digits: [7, 7, 5, 4, 0, 6, 0, 6, 0, 4]


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained CNN model
model = load_model('digit_cnn.keras')

# Load full image (no cropping)
img = cv2.imread('digit.jpg')
roi = img  # directly use the whole image

# Preprocessing
gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5,5), 0)
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Morphological closing to merge segments
kernel = np.ones((5,5), np.uint8)
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Find contours
contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

annotated = roi.copy()
detected_digits = []

for c in contours:
    area = cv2.contourArea(c)
    if area > 100:  # filter tiny noise
        x_c, y_c, w_c, h_c = cv2.boundingRect(c)
        digit_patch = thresh[y_c:y_c+h_c, x_c:x_c+w_c]

        # Preprocess for model
        d_resized = cv2.resize(digit_patch, (28,28))
        d_resized = 255 - d_resized
        d_norm = d_resized.astype('float32') / 255.0
        d_input = d_norm.reshape(1, 28, 28, 1)

        # Predict
        pred = model.predict(d_input, verbose=0)
        label = int(np.argmax(pred))
        detected_digits.append(label)

        # Draw results
        cv2.rectangle(annotated, (x_c, y_c), (x_c+w_c, y_c+h_c), (0,255,0), 2)
        cv2.putText(annotated, str(label), (x_c, y_c-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

cv2.imshow('Closed', closed)
cv2.imshow('Detected Digits', annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Detected digits:", detected_digits)


Detected digits: [6, 6, 4, 7, 2, 0, 7, 7, 9, 2, 7, 6, 7, 2, 5, 7, 6, 7, 0, 6, 0, 8, 0, 6, 7, 0, 5, 6, 6, 9, 0, 6, 0, 6, 6, 0, 6, 4, 7, 6, 6, 4, 2, 7, 4, 7, 0, 6, 6, 0, 7, 0, 6]
